In [1]:
#Reset testbed
def reset_testbed() :
    #!cd ..; cd..; sudo ./setup_testbed.sh; sudo ip netns exec ns_mid ./../../setup_ns_mid.sh;
    print("testbed is empty")
    

In [2]:
from shutil import copyfile
import os.path

def save_measurement(owd, container, duration, counter) :
    
    save_directory    = "../saves/"+container+"/csv/"
    owd_str           = "owd" + str(owd)
    veths             = ["veth0_stats","veth3_stats","veth0_aggr","veth3_aggr"]
    duration_str      = str(duration * 25)
    
    for veth in veths : 
        src_file = veth + ".csv"
        dst_file = veth + "_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/" + src_file
        dst_path = save_directory + dst_file
        copyfile(src_path, dst_path)
        
        src_file = "owd_1.csv"
        dst_file = "peer1_owd_1_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/peer1/" + src_file
        dst_path = save_directory + dst_file
        copyfile(src_path, dst_path)

        src_file = "owd_1.csv"
        dst_file = "peer2_owd_1_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/peer2/" + src_file
        dst_path = save_directory + dst_file
        if(os.path.exists(src_path)) : 
            copyfile(src_path, dst_path)
        
        src_file = "owd_1.csv"
        dst_file = "peer3_owd_1_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/peer3/" + src_file
        dst_path = save_directory + dst_file
        if(os.path.exists(src_path)) : 
            copyfile(src_path, dst_path)
        
    #PSNR metric
    #!cd ..; cd ..; vqmt source.yuv destination.yuv 352 288 $duration_str 1 results PSNR SSIM MSSSIM VIFP
    dst_path = save_directory + "psnr_result_" + owd_str + "_" + str(counter) + ".csv"
    copyfile("../../results_psnr.csv", dst_path)
    dst_path = save_directory + "ssim_result_" + owd_str + "_" + str(counter) + ".csv"
    copyfile("../../results_ssim.csv", dst_path)
    dst_path = save_directory + "msssim_result_" + owd_str + "_" + str(counter) + ".csv"
    copyfile("../../results_msssim.csv", dst_path)
    dst_path = save_directory + "vifp_result_" + owd_str + "_" + str(counter) + ".csv"
    copyfile("../../results_vifp.csv", dst_path)


In [3]:
from shutil import copyfile
import os.path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

#range_ = 1200
#owd = 100
#container = "rmcat1"
def eval_measurements(owd, container, time_range_):
    save_directory    = "../saves/"+container+"/csv/"
    owd_str           = "owd" + str(owd)
    columns = ['sender_rate','fec_rate','capacity']
    alldf = pd.DataFrame(columns=columns)
    alldf = alldf.fillna(0) # with 0s rather than NaNs
    for x in range(0, 9):
        csv_file = save_directory + "veth_stats_" + owd_str + "_" + str(x) + ".csv"
        if(os.path.isfile(csv_file) == False) :
            print('Missing file: {}'.format(csv_file))
            continue

        df = pd.read_csv(csv_file, names=columns)
        alldf = pd.concat([alldf, df[:time_range_]])
    
    print("\nAverages\n" + str(alldf.mean()))
    print("\nStandard deviations\n" + str(alldf.std()))



In [4]:
#Plotting 
def plot_rmcat1(owd, container, veth_str,labels, legends, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    path_delay_str  = str(owd * 1000)

    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat1.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_owd_1_result" + str(owd) + "_" + pdf_num_str + ".pdf"

    if(labels and legends) :
        !gnuplot -e "labels='1'" -e "throughput_file='$throughput_file'" -e "legends='1'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'"  $plot_file
    elif(labels) : 
        !gnuplot -e "labels='1'" -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'" $plot_file
    elif(legends) : 
        !gnuplot -e "legends='1'" -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'" $plot_file
    else : 
        !gnuplot -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'" $plot_file
    


In [5]:
#Plotting 
def plot_rmcat2(owd, container, veth_str, labels, legends, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    peer2_str       = "peer2"
    labels_str      = str(labels)
    legends_str     = str(legends)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file2  = "../saves/" + container + "/csv/" + peer2_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat2.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "throughput_file='$throughput_file'" -e "legends='1'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "owd_file2='$owd_file2'" $plot_file
#-e "labels='" + str(labels) + "'" -e "legends='" + str(legends) + "'"

In [105]:
plot_rmcat1(50, "rmcat1", "veth0_stats", 0, 0, 0)
#plot_rmcat2(50, "rmcat2", "veth0_stats", 1, 1, 0)

In [7]:
#Plotting 
def plot_snd_throughputs(owd, container, duration, range_, flownum, labels, legends, pdf_num, veth_str) :
    
    range_str       = str(range_)
    duration_str    = str(duration)
    flownum_str     = str(flownum)
    pdf_num_str     = str(pdf_num)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file       = "../gnuplots/snd_throughputs.plot"
    output          = "../saves/" + container + "/pdfs/" + veth_str + "_owd" + str(owd) + "_" + pdf_num_str + ".pdf"

    if(labels and legends) :
        !gnuplot -e "labels='1'" -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    elif(labels) : 
        !gnuplot -e "labels='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    elif(legends) : 
        !gnuplot -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    else : 
        !gnuplot -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file


In [8]:
#Plotting 
def plot_snd_throughputs_rmcat3(owd, container, duration, range_, labels, legends, pdf_num, veth_str) :
    
    range_str       = str(range_)
    duration_str    = str(duration)
    pdf_num_str     = str(pdf_num)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file       = "../gnuplots/snd_throughputs_rmcat3.plot"
    output          = "../saves/" + container + "/pdfs/" + veth_str + "_owd" + str(owd) + "_" + pdf_num_str + ".pdf"

    if(labels and legends) :
        !gnuplot -e "labels='1'" -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "duration='$duration_str'" $plot_file
    elif(labels) : 
        !gnuplot -e "labels='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "duration='$duration_str'" $plot_file
    elif(legends) : 
        !gnuplot -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "duration='$duration_str'" $plot_file
    else : 
        !gnuplot -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "duration='$duration_str'" $plot_file


In [9]:
#Plotting 
def plot_snd_throughputs_tcp(owd, container, duration, range_, flownum, labels, legends, pdf_num, veth_str) :
    
    range_str       = str(range_)
    duration_str    = str(duration)
    pdf_num_str     = str(pdf_num)
    flownum_str     = str(flownum)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file       = "../gnuplots/snd_throughputs_tcp.plot"
    output          = "../saves/" + container + "/pdfs/" + veth_str + "_owd" + str(owd) + "_" + pdf_num_str + ".pdf"

    if(labels and legends) :
        !gnuplot -e "labels='1'" -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    elif(labels) : 
        !gnuplot -e "labels='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    elif(legends) : 
        !gnuplot -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    else : 
        !gnuplot -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file


In [10]:
#Plotting 
def plot_owd(owd, container, duration, range_, labels, legends, pdf_num, peer) :
    
    range_str       = str(range_)
    duration_str    = str(duration)
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer" + str(peer)
    path_delay_str  = str(owd * 1000)
    
    owd_file   = "../saves/" + container + "/csv/" + peer_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/owd.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".pdf"

    if(labels and legends) :
        !gnuplot -e "labels='1'" -e "legends='1'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "range='$range_str'" -e "path_delay='$path_delay_str'" -e "duration='$duration_str'" $plot_file
    elif(labels) : 
        !gnuplot -e "labels='1'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "range='$range_str'" -e "path_delay='$path_delay_str'" -e "duration='$duration_str'" $plot_file
    elif(legends) : 
        !gnuplot -e "legends='1'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "range='$range_str'" -e "path_delay='$path_delay_str'"-e "duration='$duration_str'" $plot_file
    else : 
        !gnuplot -e "output_file='$output'" -e "owd_file='$owd_file'" -e "range='$range_str'" -e "path_delay='$path_delay_str'" -e "duration='$duration_str'" $plot_file


In [132]:
#RMCAT1 Test

container = "rmcat1"
veth0_str = "veth0_stats"
duration  = 100
range_    = 3000
flownum   = 1

plays     = [1,0,0] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_rmcat1_test.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
        
        !cd ..;cd ..; scripts2/runs/rmcat1.sh -s $owd >> log_rmcat1_test.txt
        save_measurement(owd, container, duration, y)
        plot_rmcat1(owd, container, veth0_str, label, legend, y)
        print("test for owd: " + str(owd) + " is done, counter: " + str(y))
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
         
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty
rm: cannot remove ‘scripts2/temp/peer2/*’: No such file or directory
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory
^C
test for owd: 50 is done, counter: 0


In [62]:
#RMCAT2 Test

container = "rmcat2"
veth0_str = "veth0_stats"
duration  = 120
range_    = 3000
flownum   = 2

plays     = [0,0,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_rmcat2_test.txt

for x in range(0,3) :

    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
        !cd ..;cd ..; scripts2/runs/rmcat2.sh -s $owd >> log_rmcat2_test.txt
        save_measurement(owd, container, duration, y)
        plot_rmcat2(owd, container, veth0_str, label, legend, y)
        #plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
    
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    

testbed is empty
rm: cannot remove ‘scripts2/temp/peer2/*’: No such file or directory
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory
^C

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).


In [ ]:
#RMCAT3 Test

container = "rmcat3"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 100
range_    = 3000
flownum   = 1

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_rmcat3_test.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
        !cd ..;cd ..; scripts2/runs/rmcat3.sh -s $owd >> log_rmcat3_test.txt
        save_measurement(owd, container, duration,  y)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth3_str)
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

In [150]:
#RMCAT4 Test

container = "rmcat4"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 120
range_    = 4000
flownum   = 3

plays     = [0,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_test_rmcat4.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; scripts2/runs/rmcat4.sh -s $owd >> log_test_rmcat4.txt
        save_measurement(owd, container, duration,  y)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
        
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
readOneFrame: cannot read 288 bytes from input file, unexpected EOF.

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
read

In [100]:
#RMCAT6 Test

container = "rmcat6"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 300
range_    = 2000
flownum   = 1

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_test_rmcat6.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; scripts2/runs/rmcat6.sh -s $owd >> log_test_rmcat6.txt
        save_measurement(owd, container, duration,  y)
        plot_snd_throughputs_tcp(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
        plot_owd(owd, container, duration, 500, label, legend, y, 1)
        print("test for owd: " + str(owd) + " is done, counter: " + str(y))
        
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
test for owd: 50 is done, counter: 0

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
test for owd: 150 is done, counter: 

In [160]:
#RMCAT7 Test

container = "rmcat7"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 300
range_    = 2000
flownum   = 1

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_test_rmcat7.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; scripts2/runs/rmcat7.sh -s $owd >> log_test_rmcat7.txt
        save_measurement(owd, container, duration,  y)
        plot_snd_throughputs_tcp(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
        
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
readOneFrame: cannot read 288 bytes from input file, unexpected EOF.

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
read

In [156]:
!pm-suspend

In [ ]:
import numpy
import sys

#output = sys.argv[1] # prints var1
output = "output.dat"

for x in range(0,10) :
    idle = numpy.random.exponential(10)
    !sleep $idle
    size = str(int(numpy.random.uniform(100, 10240)))
    !dd if=/dev/zero of=$output  bs="$size"KB  count=1 > log.txt
    !iperf -c 127.0.0.1 -F $output -p 1234 > log.txt

